In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
270,application_1738075734771_0271,pyspark,idle,Link,Link,None,
284,application_1738075734771_0285,pyspark,idle,Link,Link,None,
285,application_1738075734771_0286,pyspark,idle,Link,Link,None,
300,application_1738075734771_0301,pyspark,idle,Link,Link,None,
326,application_1738075734771_0327,pyspark,idle,Link,Link,None,
330,application_1738075734771_0331,pyspark,idle,Link,Link,None,
332,application_1738075734771_0333,pyspark,idle,Link,Link,None,
343,application_1738075734771_0344,pyspark,idle,Link,Link,None,
353,application_1738075734771_0354,pyspark,idle,Link,Link,None,
368,application_1738075734771_0369,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, first, regexp_replace, concat, lit, lower, year, to_date
from sedona.spark import *
import time

# Δημιουργία Spark Session
spark = SparkSession.builder \
    .appName("Query 3") \
    .getOrCreate()

# Sedona Context για GeoJSON δεδομένα
sedona = SedonaContext.create(spark)

# Μέτρηση χρόνου
start_time = time.time()

# Φόρτωση δεδομένων
crime_2010_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_2020_present_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
income_path = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv"
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
re_codes_path = "s3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv"

crime_df_2010_2019 = spark.read.csv(crime_2010_2019_path, header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv(crime_2020_present_path, header=True, inferSchema=True)
crime_df = crime_df_2010_2019.union(crime_df_2020_present)
income_df = spark.read.csv(income_path, header=True, inferSchema=True)
re_codes_df = spark.read.csv(re_codes_path, header=True, inferSchema=True)

# Φόρτωση και επεξεργασία GeoJSON δεδομένων
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

# Φιλτράρισμα των τετραγώνων του Los Angeles και επιλογή μόνο των επιθυμητών στηλών
LA_blocks = flattened_df.filter(col("CITY") == "Los Angeles") \
    .select("COMM", "HOUSING10", "POP_2010", "ZCTA10", "geometry")

# Αφαίρεση του "$" και οποιωνδήποτε κενών ή άλλων μη αριθμητικών χαρακτήρων
income_df = income_df.withColumn(
    "Estimated Median Income",
    regexp_replace(col("Estimated Median Income"), r"[^0-9.]", "").cast("double")
)

# Δημιουργία γεωμετρικών σημείων για τα εγκλήματα με φιλτράρισμα για το έτος 2015
crime_points_2015 = crime_df.filter(
    (col("LAT").isNotNull()) &
    (col("LON").isNotNull()) &
    (year(to_date(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")) == 2015)
).withColumn("geometry", ST_Point(col("LON"), col("LAT"))) \
 .select("geometry", "Vict Descent")

# Υπολογισμός κατα κεφαλήν εισοδήματος ανά περιοχή
average_income_per_person = LA_blocks.join(
    income_df,
    LA_blocks["ZCTA10"] == income_df["Zip Code"],
    "inner"
).withColumn(
    "Total Income Per Block",
    (col("HOUSING10") * col("Estimated Median Income")).alias("Total Income Per Block")
).groupBy("COMM").agg(
    ST_Union_Aggr("geometry").alias("geometry"),
    sum("Total Income Per Block").alias("Total Income"),
    sum("POP_2010").alias("Total Population")
).withColumn(
    "Avg Income Per Person ($)",
    (col("Total Income") / col("Total Population"))
).select(
    "COMM", 
    "Avg Income Per Person ($)",
    "geometry"
)

#  3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
top_3_income_blocks = average_income_per_person.orderBy("Avg Income Per Person ($)", ascending=False).limit(3)

#  3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
bottom_3_income_blocks = average_income_per_person.orderBy("Avg Income Per Person ($)", ascending=True).limit(3)

# Υπολογισμός αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
victims_in_top_3_income_blocks = crime_points_2015.join(
    top_3_income_blocks,
    ST_Within(crime_points_2015["geometry"], bottom_3_income_blocks["geometry"]),
    "inner"
).groupBy("Vict Descent").agg(
    count("*").alias("#"),
).join(
    re_codes_df,
    "Vict Descent",
    "inner"
).select(
    col("Vict Descent Full").alias("Victim Descent"), 
    "#"
).orderBy("#", ascending=False)

# Εμφάνιση αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
victims_in_top_3_income_blocks.show(truncate=False)

# Υπολογισμός αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
victims_in_bottom_3_income_blocks = crime_points_2015.join(
    bottom_3_income_blocks,
    ST_Within(crime_points_2015["geometry"], bottom_3_income_blocks["geometry"]),
    "inner"
).groupBy("Vict Descent").agg(
    count("*").alias("#"),
).join(
    re_codes_df,
    "Vict Descent",
    "inner"
).select(
    col("Vict Descent Full").alias("Victim Descent"), 
    "#"
).orderBy("#", ascending=False)

# Εμφάνιση αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
victims_in_bottom_3_income_blocks.show(truncate=False)

end_time = time.time()

# Εμφάνιση χρόνου εκτέλεσης
print(f"Configuration 1 (2 executors * 1 cores/2GB memory) Execution Time: {(end_time - start_time):.2f} seconds")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
379,application_1738075734771_0380,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+---+
|Victim Descent                |#  |
+------------------------------+---+
|White                         |695|
|Other                         |86 |
|Hispanic/Latin/Mexican        |77 |
|Unknown                       |49 |
|Black                         |43 |
|Other Asian                   |22 |
|Chinese                       |1  |
|American Indian/Alaskan Native|1  |
+------------------------------+---+

+------------------------------+----+
|Victim Descent                |#   |
+------------------------------+----+
|Hispanic/Latin/Mexican        |3342|
|Black                         |1127|
|White                         |428 |
|Other                         |252 |
|Other Asian                   |138 |
|Unknown                       |30  |
|American Indian/Alaskan Native|23  |
|Korean                        |4   |
|Filipino                      |3   |
|Chinese                       |3   |
|Guamanian                     |1   |
|AsianIndian           

In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2",
        "spark.driver.memory": "2g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
270,application_1738075734771_0271,pyspark,idle,Link,Link,None,
284,application_1738075734771_0285,pyspark,idle,Link,Link,None,
285,application_1738075734771_0286,pyspark,idle,Link,Link,None,
300,application_1738075734771_0301,pyspark,idle,Link,Link,None,
326,application_1738075734771_0327,pyspark,idle,Link,Link,None,
330,application_1738075734771_0331,pyspark,idle,Link,Link,None,
332,application_1738075734771_0333,pyspark,idle,Link,Link,None,
334,application_1738075734771_0335,pyspark,idle,Link,Link,None,
343,application_1738075734771_0344,pyspark,idle,Link,Link,None,
353,application_1738075734771_0354,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, first, regexp_replace, concat, lit, lower, year, to_date
from sedona.spark import *
import time

# Δημιουργία Spark Session
spark = SparkSession.builder \
    .appName("Query 3") \
    .getOrCreate()

# Sedona Context για GeoJSON δεδομένα
sedona = SedonaContext.create(spark)

# Μέτρηση χρόνου
start_time = time.time()

# Φόρτωση δεδομένων
crime_2010_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_2020_present_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
income_path = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv"
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
re_codes_path = "s3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv"

crime_df_2010_2019 = spark.read.csv(crime_2010_2019_path, header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv(crime_2020_present_path, header=True, inferSchema=True)
crime_df = crime_df_2010_2019.union(crime_df_2020_present)
income_df = spark.read.csv(income_path, header=True, inferSchema=True)
re_codes_df = spark.read.csv(re_codes_path, header=True, inferSchema=True)

# Φόρτωση και επεξεργασία GeoJSON δεδομένων
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

# Φιλτράρισμα των τετραγώνων του Los Angeles και επιλογή μόνο των επιθυμητών στηλών
LA_blocks = flattened_df.filter(col("CITY") == "Los Angeles") \
    .select("COMM", "HOUSING10", "POP_2010", "ZCTA10", "geometry")

# Αφαίρεση του "$" και οποιωνδήποτε κενών ή άλλων μη αριθμητικών χαρακτήρων
income_df = income_df.withColumn(
    "Estimated Median Income",
    regexp_replace(col("Estimated Median Income"), r"[^0-9.]", "").cast("double")
)

# Δημιουργία γεωμετρικών σημείων για τα εγκλήματα με φιλτράρισμα για το έτος 2015
crime_points_2015 = crime_df.filter(
    (col("LAT").isNotNull()) &
    (col("LON").isNotNull()) &
    (year(to_date(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")) == 2015)
).withColumn("geometry", ST_Point(col("LON"), col("LAT"))) \
 .select("geometry", "Vict Descent")

# Υπολογισμός κατα κεφαλήν εισοδήματος ανά περιοχή
average_income_per_person = LA_blocks.join(
    income_df,
    LA_blocks["ZCTA10"] == income_df["Zip Code"],
    "inner"
).withColumn(
    "Total Income Per Block",
    (col("HOUSING10") * col("Estimated Median Income")).alias("Total Income Per Block")
).groupBy("COMM").agg(
    ST_Union_Aggr("geometry").alias("geometry"),
    sum("Total Income Per Block").alias("Total Income"),
    sum("POP_2010").alias("Total Population")
).withColumn(
    "Avg Income Per Person ($)",
    (col("Total Income") / col("Total Population"))
).select(
    "COMM", 
    "Avg Income Per Person ($)",
    "geometry"
)

#  3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
top_3_income_blocks = average_income_per_person.orderBy("Avg Income Per Person ($)", ascending=False).limit(3)

#  3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
bottom_3_income_blocks = average_income_per_person.orderBy("Avg Income Per Person ($)", ascending=True).limit(3)

# Υπολογισμός αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
victims_in_top_3_income_blocks = crime_points_2015.join(
    top_3_income_blocks,
    ST_Within(crime_points_2015["geometry"], bottom_3_income_blocks["geometry"]),
    "inner"
).groupBy("Vict Descent").agg(
    count("*").alias("#"),
).join(
    re_codes_df,
    "Vict Descent",
    "inner"
).select(
    col("Vict Descent Full").alias("Victim Descent"), 
    "#"
).orderBy("#", ascending=False)

# Εμφάνιση αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
victims_in_top_3_income_blocks.show(truncate=False)

# Υπολογισμός αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
victims_in_bottom_3_income_blocks = crime_points_2015.join(
    bottom_3_income_blocks,
    ST_Within(crime_points_2015["geometry"], bottom_3_income_blocks["geometry"]),
    "inner"
).groupBy("Vict Descent").agg(
    count("*").alias("#"),
).join(
    re_codes_df,
    "Vict Descent",
    "inner"
).select(
    col("Vict Descent Full").alias("Victim Descent"), 
    "#"
).orderBy("#", ascending=False)

# Εμφάνιση αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
victims_in_bottom_3_income_blocks.show(truncate=False)

end_time = time.time()

# Εμφάνιση χρόνου εκτέλεσης
print(f"Configuration 2 (2 executors * 2 cores/4GB memory) Execution Time: {(end_time - start_time):.2f} seconds")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
364,application_1738075734771_0365,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+---+
|Victim Descent                |#  |
+------------------------------+---+
|White                         |695|
|Other                         |86 |
|Hispanic/Latin/Mexican        |77 |
|Unknown                       |49 |
|Black                         |43 |
|Other Asian                   |22 |
|Chinese                       |1  |
|American Indian/Alaskan Native|1  |
+------------------------------+---+

+------------------------------+----+
|Victim Descent                |#   |
+------------------------------+----+
|Hispanic/Latin/Mexican        |3342|
|Black                         |1127|
|White                         |428 |
|Other                         |252 |
|Other Asian                   |138 |
|Unknown                       |30  |
|American Indian/Alaskan Native|23  |
|Korean                        |4   |
|Filipino                      |3   |
|Chinese                       |3   |
|AsianIndian                   |1   |
|Guamanian             

In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
270,application_1738075734771_0271,pyspark,idle,Link,Link,None,
284,application_1738075734771_0285,pyspark,idle,Link,Link,None,
285,application_1738075734771_0286,pyspark,idle,Link,Link,None,
300,application_1738075734771_0301,pyspark,idle,Link,Link,None,
326,application_1738075734771_0327,pyspark,idle,Link,Link,None,
330,application_1738075734771_0331,pyspark,idle,Link,Link,None,
332,application_1738075734771_0333,pyspark,idle,Link,Link,None,
334,application_1738075734771_0335,pyspark,idle,Link,Link,None,
343,application_1738075734771_0344,pyspark,idle,Link,Link,None,
353,application_1738075734771_0354,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, first, regexp_replace, concat, lit, lower, year, to_date
from sedona.spark import *
import time

# Δημιουργία Spark Session
spark = SparkSession.builder \
    .appName("Query 3") \
    .getOrCreate()

# Sedona Context για GeoJSON δεδομένα
sedona = SedonaContext.create(spark)

# Μέτρηση χρόνου
start_time = time.time()

# Φόρτωση δεδομένων
crime_2010_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_2020_present_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
income_path = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv"
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
re_codes_path = "s3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv"

crime_df_2010_2019 = spark.read.csv(crime_2010_2019_path, header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv(crime_2020_present_path, header=True, inferSchema=True)
crime_df = crime_df_2010_2019.union(crime_df_2020_present)
income_df = spark.read.csv(income_path, header=True, inferSchema=True)
re_codes_df = spark.read.csv(re_codes_path, header=True, inferSchema=True)

# Φόρτωση και επεξεργασία GeoJSON δεδομένων
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

# Φιλτράρισμα των τετραγώνων του Los Angeles και επιλογή μόνο των επιθυμητών στηλών
LA_blocks = flattened_df.filter(col("CITY") == "Los Angeles") \
    .select("COMM", "HOUSING10", "POP_2010", "ZCTA10", "geometry")

# Αφαίρεση του "$" και οποιωνδήποτε κενών ή άλλων μη αριθμητικών χαρακτήρων
income_df = income_df.withColumn(
    "Estimated Median Income",
    regexp_replace(col("Estimated Median Income"), r"[^0-9.]", "").cast("double")
)

# Δημιουργία γεωμετρικών σημείων για τα εγκλήματα με φιλτράρισμα για το έτος 2015
crime_points_2015 = crime_df.filter(
    (col("LAT").isNotNull()) &
    (col("LON").isNotNull()) &
    (year(to_date(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")) == 2015)
).withColumn("geometry", ST_Point(col("LON"), col("LAT"))) \
 .select("geometry", "Vict Descent")

# Υπολογισμός κατα κεφαλήν εισοδήματος ανά περιοχή
average_income_per_person = LA_blocks.join(
    income_df,
    LA_blocks["ZCTA10"] == income_df["Zip Code"],
    "inner"
).withColumn(
    "Total Income Per Block",
    (col("HOUSING10") * col("Estimated Median Income")).alias("Total Income Per Block")
).groupBy("COMM").agg(
    ST_Union_Aggr("geometry").alias("geometry"),
    sum("Total Income Per Block").alias("Total Income"),
    sum("POP_2010").alias("Total Population")
).withColumn(
    "Avg Income Per Person ($)",
    (col("Total Income") / col("Total Population"))
).select(
    "COMM", 
    "Avg Income Per Person ($)",
    "geometry"
)

#  3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
top_3_income_blocks = average_income_per_person.orderBy("Avg Income Per Person ($)", ascending=False).limit(3)

#  3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
bottom_3_income_blocks = average_income_per_person.orderBy("Avg Income Per Person ($)", ascending=True).limit(3)

# Υπολογισμός αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
victims_in_top_3_income_blocks = crime_points_2015.join(
    top_3_income_blocks,
    ST_Within(crime_points_2015["geometry"], bottom_3_income_blocks["geometry"]),
    "inner"
).groupBy("Vict Descent").agg(
    count("*").alias("#"),
).join(
    re_codes_df,
    "Vict Descent",
    "inner"
).select(
    col("Vict Descent Full").alias("Victim Descent"), 
    "#"
).orderBy("#", ascending=False)

# Εμφάνιση αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το υψηλότερο κατά κεφαλήν εισόδημα
victims_in_top_3_income_blocks.show(truncate=False)

# Υπολογισμός αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
victims_in_bottom_3_income_blocks = crime_points_2015.join(
    bottom_3_income_blocks,
    ST_Within(crime_points_2015["geometry"], bottom_3_income_blocks["geometry"]),
    "inner"
).groupBy("Vict Descent").agg(
    count("*").alias("#"),
).join(
    re_codes_df,
    "Vict Descent",
    "inner"
).select(
    col("Vict Descent Full").alias("Victim Descent"), 
    "#"
).orderBy("#", ascending=False)

# Εμφάνιση αριθμού θυμάτων ανά φυλετικό γκρουπ για τις 3 περιοχές με το χαμηλότερο κατά κεφαλήν εισόδημα
victims_in_bottom_3_income_blocks.show(truncate=False)

end_time = time.time()

# Εμφάνιση χρόνου εκτέλεσης
print(f"Configuration 3 (2 executors * 4 cores/8GB memory) Execution Time: {(end_time - start_time):.2f} seconds")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
367,application_1738075734771_0368,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+---+
|Victim Descent                |#  |
+------------------------------+---+
|White                         |695|
|Other                         |86 |
|Hispanic/Latin/Mexican        |77 |
|Unknown                       |49 |
|Black                         |43 |
|Other Asian                   |22 |
|American Indian/Alaskan Native|1  |
|Chinese                       |1  |
+------------------------------+---+

+------------------------------+----+
|Victim Descent                |#   |
+------------------------------+----+
|Hispanic/Latin/Mexican        |3342|
|Black                         |1127|
|White                         |428 |
|Other                         |252 |
|Other Asian                   |138 |
|Unknown                       |30  |
|American Indian/Alaskan Native|23  |
|Korean                        |4   |
|Filipino                      |3   |
|Chinese                       |3   |
|AsianIndian                   |1   |
|Guamanian             